In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

/home/oumar/Projects/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
df = pd.read_csv("cleaned_data.csv")
df.head()

,title,body
0,I was once reading this article that went as: ...,"['#SportsAnalytics', 'Im a ManU fan. I want Ca..."
1,How do you change your setup on Jupyter to loo...,['IMPORT EXCEL DATAFRAME']
2,The reality of 9/10 data scientist job specs!,['It would be Requirements-PhD and 15 years ex...
3,oh fuck this had no business to be funny,"[""God the fact that he didn't even refence ano..."
4,ARtYfIshaL NtEliGeNtS,"['Read this as ""artificial negligence"" and it ..."


In [ ]:
model_id = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

NameError: name 'AutoTokenizer' is not defined

In [9]:
# Prepare data for finetuning
from datasets import Dataset

data = Dataset.from_pandas(df)
print(data[0])

{'title': 'I was once reading this article that went as: The AI already predicted how many goals Cavani will score at Manchester United. It was a linear regression.', 'body': "['#SportsAnalytics', 'Im a ManU fan. I want Cavani to do well. But Im also finishing up my Masters in Data Analytics and this made me laugh out loud. Well done', 'tbf im pretty sure a linear regression could predict Lewis Hammertime as a 8 time world champion. Should I write an article and call it AI?']"}


In [ ]:
data = data.train_test_split(test_size=0.2)


In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['title', 'body'],
        num_rows: 3955
    })
    test: Dataset({
        features: ['title', 'body'],
        num_rows: 989
    })
})

In [13]:
assistant = "You are a datascience tutor. Users will ask you you data science questions and you will provide them with the answers."

def format_data(examples):
    inputs = [assistant + " " + doc for doc in examples["title"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(target = examples["body"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = data.map(format_data, batched=True)

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]


NameError: name 'tokenizer' is not defined

In [ ]:
# Evaluation
import evaluate
import nltk
import numpy as np



nltk.download('punkt')
metric = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [16]:
# parameters
learning_rate = 3e-4
batch_size = 4
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
weight_decay = 0.01
num_epochs = 5

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_dir="./logs",
)

NameError: name 'Seq2SeqTrainingArguments' is not defined

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)